#FastSpeech model research

##Download repository

In [ ]:
import os

if not os.path.isdir("/content/FastSpeech2/"):
  !git clone https://github.com/murzinov01/FastSpeech2

Cloning into 'FastSpeech2'...
remote: Enumerating objects: 738, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 738 (delta 28), reused 8 (delta 4), pack-reused 679
Receiving objects: 100% (738/738), 290.37 MiB | 46.96 MiB/s, done.
Resolving deltas: 100% (213/213), done.


In [ ]:
%cd FastSpeech2/

/content/FastSpeech2


In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 3.1MB 3.2MB/s 
     |████████████████████████████████| 1.6MB 27.5MB/s 
     |████████████████████████████████| 3.6MB 34.0MB/s 
     |████████████████████████████████| 14.6MB 164kB/s 
     |████████████████████████████████| 788kB 63.7MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
     |████████████████████████████████| 645kB 53.9MB/s 
     |████████████████████████████████| 6.8MB 67.5MB/s 
     |████████████████████████████████| 25.9MB 1.5MB/s 
     |████████████████████████████████| 3.0MB 57.3MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 245kB 67.1MB/s 
     |████████████████████████████████| 184kB 59.3MB/s 
     |████████████████████████████████| 20.2MB 95.6MB/s 
  Created wheel for librosa: filename=librosa-0.7.2-cp37-none-any.whl size=1612903 sha256=e53593319786fe7f7d50f90b187653b05dfcdbdb87251cd89b38b49472a649ce
  Store

###Mounting google disk

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Download M-AILABS Russian dataset

In [ ]:
%cd /content/drive/MyDrive/
!wget http://www.caito.de/data/Training/stt_tts/ru_RU.tgz
!tar -zxvf ru_RU.tgz
# # Rename directory
!mv ru_RU M_AILABS_Ru

### Preprocess Dataset


In [ ]:
#import random

In [ ]:
# """
# Move all wav file to the common folder
# Concat all text files into one
# """
# def refactor_M_AILABS(sex='female', speaker_index=0
#                       , dataset_dir="/content/drive/MyDrive/M_AILABS_Ru/"
#                       , out_dir="/content/drive/MyDrive/M_AILABS_Ru/"
#                       , move_files=True
#                       ):
#   print(" *** Start refactoring... ***")
#   path = f"{dataset_dir}{''if dataset_dir[-1] == '/' else '/'}by_book/{sex}"
#   %cd $path
#   speakers = !ls
#   books_path = f"{path}/{speakers[speaker_index]}"
#   %cd $books_path
#   books_folders = !ls -d */ | cut -f1 -d'/'
#   path = f"{out_dir}{''if out_dir[-1] == '/' else '/'}"
#   %cd $path
#   !mkdir wavs
#   wavs_path = f"{path}wavs/"
#   !touch mlbs_audio_text.txt
#   for book in books_folders:
#     cur_wavs_path = f"{books_path}/{book}/wavs/*"
#     cur_text_path = f"{books_path}/{book}/metadata.csv"
#     !cat $cur_text_path >> mlbs_audio_text.txt
#     if move_files:
#       !mv -v $cur_wavs_path $wavs_path
#   print(" *** Refactoring finished! ***")

In [ ]:
#refactor_M_AILABS()

In [ ]:
#!rm -rf by_book && mv mlbs_audio_text.txt metadata.csv

##Preprocessing Data using Montreal Forced Aligner

In [ ]:
%cd /content/FastSpeech2

/content/FastSpeech2


In [ ]:
!python3 prepare_align.py config/M_AILABS_Ru/preprocess.yaml

In [ ]:
!cp -R ./wav /content/drive/MyDrive/wav

###Install Miniconda

In [ ]:
%%writefile install_mfa.sh
#!/bin/bash

## a script to install Montreal Forced Aligner (MFA)

root_dir=${1:-/tmp/mfa}
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

# create py38 env
$root_dir/miniconda3/bin/conda create -n aligner -c conda-forge openblas python=3.8 openfst pynini ngram baumwelch -y
source $root_dir/miniconda3/bin/activate aligner

# install mfa, download kaldi
pip install montreal-forced-aligner # install requirements
pip install git+https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner.git # install latest updates

mfa thirdparty download

echo -e "\n======== DONE =========="
echo -e "\nTo activate MFA, run: source $root_dir/miniconda3/bin/activate aligner"
echo -e "\nTo delete MFA, run: rm -rf $root_dir"
echo -e "\nSee: https://montreal-forced-aligner.readthedocs.io/en/latest/aligning.html to know how to use MFA"

Writing install_mfa.sh


###Download and install mfa

In [ ]:
INSTALL_DIR="/tmp/mfa" # path to install directory
!bash ./install_mfa.sh {INSTALL_DIR}
!source {INSTALL_DIR}/miniconda3/bin/activate aligner;

Miniconda3-latest-L 100%[===================>]  89.87M   134MB/s    in 0.7s    
PREFIX=/tmp/mfa/miniconda3
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /tmp/mfa/miniconda3

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py38h27cfd23_1003
    - ca-certificates==2020.10.14=0
    - certifi==2020.6.20=pyhd3eb1b0_3
    - cffi==1.14.3=py38h261ae71_2
    - chardet==3.0.4=py38h06a4308_1003
    - conda-package-handling==1.7.2=py38h03888b9_0
    - conda==4.9.2=py38h06a4308_0
    - cryptography==3.2.1=py38h3c74f83_1
    - idna==2.10=py_0
    - ld_impl_linux-64==2.33.1=h53a641e_7
    - libedit==3.1.20191231=h14c3975_1
    - libffi==3.3=he6710b0_2
    - libgcc-ng==9.1.0=hdf63c60_0
    - libstdcxx-ng==9.1.0=hdf63c60_0
    - ncurses==6.2=he6710b0_1
    - openssl==1.1.1h=h7b6447c_0
    - pip==20.2.4=py38h06a4308_0
    - pycosat==0.6.3=py38h7b6447c_1
    - pycparser==2.20=py_2
    - pyopenssl==19.1.0=pyhd3eb1b0_1
    

In [ ]:
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; mfa align --help

usage: mfa align
       [-h]
       [--config_path CONFIG_PATH]
       [-s SPEAKER_CHARACTERS]
       [-t TEMP_DIRECTORY]
       [-j NUM_JOBS]
       [-v]
       [-c]
       [-d]
       corpus_directory
       dictionary_path
       acoustic_model_path
       output_directory

positional arguments:
  corpus_directory
    Full path
    to the
    directory
    to align
  dictionary_path
    Full path
    to the pron
    unciation
    dictionary
    to use
  acoustic_model_path
    Full path
    to the
    archive
    containing
    pre-trained
    model or
    language ()
  output_directory
    Full path
    to output
    directory,
    will be
    created if
    it doesn't
    exist

optional arguments:
  -h, --help
    show this
    help
    message and
    exit
  --config_path CONFIG_PATH
    Path to
    config file
    to use for
    alignment
  -s SPEAKER_CHARACTERS, --speaker_characters SPEAKER_CHARACTERS
    Number of
    characters
    of file
    names to
    use for
    determ

###Install sox tool to convert to 16k audio clips

In [ ]:
!sudo apt install -q -y sox
!mkdir ./wav_audio
!echo "normalize audio clips to sample rate of 16k"
!find /content/drive/MyDrive/raw_data/M_AILABS_Ru/ -name "*.wav" -type f -execdir sox --norm=-3 {} -r 16k -c 1 `pwd`/wav_audio/{} \;
!echo "Number of clips" $(ls ./wav_audio/ | wc -l)

Reading package lists...
Building dependency tree...
Reading state information...
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
mkdir: cannot create directory ‘./wav_audio’: File exists
normalize audio clips to sample rate of 16k
Number of clips 7305


In [ ]:
# !mkdir -p ./wav_audio
# !cp -R /content/drive/MyDrive/wav_audio ./wav_audio

^C


###Create transcript files from metadata.csv

In [ ]:
lines = open('/content/drive/MyDrive/M_AILABS_Ru/metadata.csv', 'r').readlines()
from tqdm.auto import tqdm
for line in tqdm(lines):
  fn, _, transcript = line.strip().split('|')
  ident = fn
  open(f'/content/drive/MyDrive/wav_audio/{ident}.txt', 'w').write(transcript)

###Create dictionary

In [ ]:
# %cd /content/
# !git clone https://github.com/nsu-ai/russian_g2p.git\

In [ ]:
# !cd russian_g2p && pip3 install -r requirements.txt

In [ ]:
# text = list()
# with open("/content/drive/MyDrive/M_AILABS_Ru/metadata.csv", 'r', encoding='utf-8') as file:
#   for raw in file:
#     str_ = raw.split("|")[1]
#     new_str = []
#     for char in str_:
#       # num_char = ord(char)
#       # print(num_char)
#       #if char not in ('0','1','2','3','4','5','6','7','8','9'):
#       #if not (65 <= num_char <= 90 or 97 <= num_char <= 122):
#       if char in "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя .,?!":
#         new_str.append(char) 
#     text.append(''.join(new_str))
# text =  "".join(text)
# text = "".join((char if char.isalpha() else " ") for char in text).split()
# text = set(text)
# # for el in text:
# #   print(el)
# with open("/content/drive/MyDrive/M_AILABS_Ru/whole_text.txt", 'w', encoding='utf-8') as file:
#   for line in text:
#     file.write(line + "\n")

In [ ]:
# !cd russian_g2p && python create_phonetic_dict.py --src /content/drive/MyDrive/M_AILABS_Ru/whole_text.txt --dst '/content/drive/MyDrive/M_AILABS_Ru/created_phonetic_dictionary.txt' --bad '/content/drive/MyDrive/M_AILABS_Ru/unknown_words.txt'

In [ ]:
# !cp /content/drive/MyDrive/M_AILABS_Ru/created_phonetic_dictionary.txt /content/drive/MyDrive/russian_lexicon.txt

In [ ]:
%cd /content/FastSpeech2/

/content/FastSpeech2


###Download a pretrained russian acoustic model, and create russian lexicon

In [ ]:
!wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/master/acoustic/russian.zip
#!wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/master/g2p/russian_g2p.zip

russian.zip         100%[===================>]  14.00M  20.9MB/s    in 0.7s    


### Train acustic model

In [ ]:
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa train_g2p /content/drive/MyDrive/russian_lexicon.txt russian_test_model.zip

Calculating alignments...
100% 25/25 [11:30<00:00, 27.61s/it]
Saved model to russian_test_model.zip


### Generate dictionary

In [ ]:
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa g2p russian_test_model.zip /content/drive/MyDrive/M_AILABS_Ru /content/drive/MyDrive/russian_dict.txt

Generating pronunciations from G2P model
Generating transcriptions for the 0 word types found in the corpus...
Generating pronunciations...
Processed 0 in 2.288818359375e-05 seconds


###Finally, align phonemes and speech

In [ ]:
!mkdir -p /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru

In [ ]:
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa train  /content/drive/MyDrive/ /content/drive/MyDrive/russian_lexicon.txt /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru
!source {INSTALL_DIR}/miniconda3/bin/activate aligner; \
mfa align -t ./temp -c -j 4 ./wav_audio /content/drive/MyDrive/russian_lexicon.txt ./russian_test_model.zip /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru

#mfa align -t ./temp -c -j 4 ./wav /content/drive/MyDrive/russian_lexicon.txt ./english.zip ./ljs_aligned

All required kaldi binaries were found!
/root/Documents/MFA/MyDrive/train_and_align.log
INFO - Setting up corpus information...
WARNING - Some issues parsing the corpus were detected. Please run the validator to get more information.
INFO - Number of speakers in corpus: 2, average number of utterances per speaker: 7304.5
INFO - Parsing dictionary without pronunciation probabilities without silence probabilities
INFO - Creating dictionary information...
INFO - Setting up training data...
Generating base features (mfcc)...
Calculating CMVN...
INFO - Initializing training for mono...
INFO - Initialization complete!
100% 39/39 [40:28<00:00, 62.28s/it]
INFO - Training complete!
INFO - Generating alignments using mono models using 5000 utterances...
INFO - Initializing training for tri...
INFO - Initialization complete!
100% 34/34 [14:55<00:00, 26.33s/it]
INFO - Training complete!
INFO - Generating alignments using tri models using 10000 utterances...
INFO - Initializing training for lda...


In [ ]:
!ls /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru

raw_data  unaligned.txt  wav_audio


In [ ]:
!cd /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru && rename 's/^......//' *.TextGrid

In [ ]:
# !cat /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru/chetvero_nischih_s000107.TextGrid
# chetvero-nischih-s000107

In [ ]:
!cd /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru && rename 'y/-/_/' *.TextGrid

In [ ]:
!mv /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/wavs  /content/drive/MyDrive/my_preprocessed_data/M_AILABS_Ru/TextGrid/M_AILABS_Ru

In [ ]:
!python3 preprocess.py config/M_AILABS_Ru/preprocess.yaml

Processing Data ...
100% 1/1 [50:42<00:00, 3042.09s/it]
Computing statistic quantities ...
Total time: 13.603439455782313 hours


In [ ]:
!ls /content/drive/MyDrive/M_AILABS_Ru/

M_AILABS_Ru


In [ ]:
!mv /content/drive/MyDrive/M_AILABS_Ru/unknown_words.txt  /content/drive/MyDrive/MAIN_AILABS_Ru_

In [ ]:
!mv /content/drive/MyDrive/M_AILABS_Ru/metadata.csv /content/drive/MyDrive/

In [ ]:
#!cd ./preprocessed_data/LJSpeech/TextGrid/LJSpeech && rename 's/^....//' *.TextGrid
#!cp -R ./preprocessed_data/M_AILABS_Ru /content/drive/MyDrive/my_preprocessed_data

###Preprocess data

In [ ]:
#!python3 preprocess.py config/M_AILABS_Ru/preprocess.yaml

In [ ]:
!unzip ./hifigan/generator_LJSpeech.pth.tar -d ./hifigan

Archive:  ./hifigan/generator_LJSpeech.pth.tar.zip
  inflating: ./hifigan/generator_LJSpeech.pth.tar  


##Train

In [ ]:
!python3 train.py -p config/M_AILABS_Ru/preprocess.yaml -m config/M_AILABS_Ru/model.yaml -t config/M_AILABS_Ru/train.yaml

In [ ]:
!git pull

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/murzinov01/FastSpeech2
   3b02d0a..77f7e18  master     -> origin/master
Updating 3b02d0a..77f7e18
Fast-forward
 config/M_AILABS_Ru/model.yaml | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)


##Inference

In [ ]:
!python3 synthesize.py \
--text "Текст, который нужно синтезировать" \
--restore_step 900000 \
--mode single \
-p config/M_AILABS_Ru/preprocess.yaml \
-m config/M_AILABS_Ru/model.yaml \
-t config/M_AILABS_Ru/train.yaml


In [ ]:
!cd output/result/M_AILABS_Ru/ && ls

In [ ]:
from IPython.display import Image
Image('output/result/M_AILABS_Ru/Текст, который нужно синтезировать.png')